In [1]:
## For debbugging
%load_ext autoreload

In [2]:
## for debugging:
%aimport mlp.keyword
%aimport mlp.tools
%autoreload 1
from mlp.keyword import *

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

import spacy, textacy, platform
print(
    f'Textacy: \t{textacy.__version__:}'+\
    f'\nspaCy: \t{spacy.__version__:>13}'+\
    f'\nPython: \t{platform.python_version()}'
    )

Textacy: 	0.4.1
spaCy: 	       1.10.0
Python: 	3.6.3


In [14]:
data_directory = os.path.join('./data', 'gs_data')
nl_col = {
    "Description": 5,
    "Resolution": 6
}

#TODO
# meta_col = {

# }

kex_gs = KeywordExtractor('Maintenance_All_Clean.xlsx',
                       nlp_cols=nl_col,
#                       meta_cols = meta_col,
                       special_replace={' -': '; '},
                       wdir=data_directory,
                       keep_temp_files=False
                      )

/home/tbsexton/Documents/Projects/DiagnosticKB/MaintenanceLanguageProcessing/mlp/keyword.py:98: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  **default_pd_kws)
921it [00:00, 9203.03it/s]

{' -': '; '}
making new file...


3435it [00:00, 8583.06it/s]
/home/tbsexton/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
0it [00:00, ?it/s]

making new file...
making new file...


3435it [00:00, 8417.16it/s]


Replacing NaN
Replacing NaN
Replacing NaN
Replacing NaN


In [15]:
# what we extracted...we'll be using 'RawText'
kex_gs.df.head()

,Description,Resolution,RawText
0,Broken bar feeder chain,Repaired,break bar_feeder chain_repair
1,No power,Replaced pin in pendant and powered machine; P...,no_power replace pin in pendant and power mach...
2,Smartscope harness broken,Parts ordered / Tech repaired,smartscope harness break part order tech repair
3,Check / Charge Accumulators,Where OK,check_charge accumulator where ok
4,Hyd leak at saw atachment,Replaced seal in saw attachment but still leak...,hyd_leak at saw atachment replace seal in saw_...


In [16]:
# use this to make a properly formatted thesaurus file for human-annotation
kex_gs.gen_vocab('new_tag_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv


In [17]:
# "fit" a thesaurus to the data
kex_gs.fit(vocab='tag_vocab.csv')
kex_gs.vocab.head(n=7)

,NE,alias,note
token,,,
replace,S,replace,NaN
unit,I,unit,NaN
motor,I,motor,NaN
spindle,I,spindle,NaN
leak,P,leak,NaN
valve,I,valve,NaN
replaced,S,replace,NaN


In [183]:
# tag the data, saving to a new DF (instead of internally)
df_pred = kex_gs.transform(save=False)
df_pred.head()

100%|██████████| 3431/3431 [00:05<00:00, 640.86it/s]


,RawText,Items,Problem,Solution,UK_tok
0,break bar_feeder chain_repair,,break,,"chain_repair, bar_feeder chain_repair, bar_feeder"
1,no_power replace pin in pendant and power mach...,"machine, cable, pin, pendant","power, short",replace,no_power
2,smartscope harness break part order tech repair,tech,break,"order, repair","harness, smartscope"
3,check_charge accumulator where ok,accumulator,,,check_charge
4,hyd_leak at saw atachment replace seal in saw_...,"saw, seal",leak,"replace seal, replace","ml, hyd_leak, saw_attachment, reapir, but_still"


In [182]:
# %%timeit
# cat_vocab = set_index(['NE'], append=True).swaplevel().sort_index(0)
# for clf, clf_thes in kex_gs.vocab.groupby('NE'):
#     print('now looping the ', clf, "dataframe")
#     pass

cat_vocab = kex_gs.vocab.groupby("NE")

tokens = ['accumulator pump', 'pump', 'remove']
vocab_list = set(kex_gs.vocab.index)

# %timeit cat_vocab.apply(lambda x: x[np.isin(x.index.values, tokens)])
# cat_vocab.apply(lambda x: x[np.isin(x.index.values, tokens)])
# %timeit kex_gs.vocab.loc[ set(tokens) & vocab_list].groupby('NE').groups
# kex_gs.vocab.loc[ set(tokens) & vocab_list].groupby('NE').groups['I'].tolist()
[i for i in tokens if not np.any(np.isin(vocab_list, i))]
np.isin(list(vocab_list), 'accumulator pump'.split(' ')).sum()

2

In [52]:
# tag-complete work-orders?
print(df_pred[df_pred.UK_tok ==''].shape[0])
df_pred[df_pred.UK_tok ==''].head()

64


,RawText,Items,Problem,Solution,UK_tok
11,spindle carrier not indexing over feed,"spindle, carrier, spindle carrier, feed",index,,
38,alarm 765053 bar clamp pressure still present ...,"bar, clamp, pressure, bar clamp, sp#6, clamp p...",alarm,clear,
66,stop switch s120.2 not release switch replace,switch,,"release, replace",
113,iemca loading failure check and charge accumul...,"iemca, accumulator",failure,"charge accumulator, charge, check",
320,solenoid stick clean down shaft with wd-40,"solenoid, shaft, stick",,clean,


In [53]:
# tag-empty? 
print(df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].shape[0])
df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].head()

60


,RawText,Items,Problem,Solution,UK_tok
610,accumulator_check be ok at_30bar,,,,"accumulator_check, ok at_30bar, at_30bar"
613,mark on od of part polished jaws operator_level,,,,"mark, operator_level, jaws, polished"
631,unload automation not return,,,,"automation, unload"
710,bar_end be find in production pan,,,,"find, pan, bar_end"
877,part_catcher will_not return in_auto,,,,"will_not return, part_catcher, in_auto, will_not"


## HVAC Data

In [31]:
data_directory = os.path.join('./data', 'hvac_data')

nl_col = {
    "Description": 5,
    "Resolution": 248
}

meta_col = {
    "AssetID": 6,
    "Location": 7,
    "Assigned": 10,
    "Reported": 43,
    "SolveDate": 11,
    "ReportDate": 44
}

kex_hvac = KeywordExtractor('hvac_raw.xlsx',
                       nlp_cols=nl_col,
                       meta_cols = meta_col,
                       special_replace={'Action Taken:': ''},
                       wdir=data_directory,
                       keep_temp_files=True
                      )

456it [00:00, 4556.69it/s]

{'Action Taken:': ''}
making new file...


16986it [00:07, 2271.13it/s]


making new file...


/home/tbsexton/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
469it [00:00, 4687.65it/s]

making new file...


16986it [00:07, 2402.24it/s]


In [32]:
# what we extracted...we'll be using 'RawText'
kex_hvac.df.head()

,Description,AssetID,Location,Assigned,SolveDate,Reported,ReportDate,Resolution,RawText
0,CHARGE ALL M/S AND PM'S TO THIS ORDER,NaN,GAITHER,FEDERLIN,2009-04-02 14:15:38,LEM801,2009-02-12 14:27:33,"for site , other that AML Action Taken:Complete.",charge m_s pm order site aml complete
1,CHARGE ALL M/S AND PM'S TO THIS ORDER FOR THE AML,NaN,215-1,FEDERLIN,2009-04-02 14:15:39,LEM801,2009-02-12 14:44:55,Action Taken: Time charged to cover PM's as ne...,charge m_s pm order aml time charge cover pm '...
2,"ADJUST,REPAI OR REPLACE FREEZE STAT",301 HV-4,301-B184,FEDERLIN,2009-04-02 14:15:41,HWRJR,2009-02-15 23:25:01,Action Taken:Reset POC Ed,adjust repai replace freeze_stat reset poc_ed
3,"RESET,ADJUST OR REPAIR FREEZE STAT",304 ACU-1,304-200,FEDERLIN,2009-04-02 14:15:42,HWRJR,2009-02-17 04:20:08,Action Taken: restart POC Ed,reset adjust repair freeze_stat restart poc_ed
4,"RSET,ADJUST OR REPAIR FREEZE STAT",220 ACU-8,220-400,FEDERLIN,2009-06-25 08:17:33,HWRJR,2009-02-17 04:22:38,reset freezestat,rset adjust repair freeze_stat reset_freezestat


In [33]:
# use this to make a properly formatted thesaurus file for human-annotation
# kex.gen_vocab('tag_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv


In [34]:
# "fit" a thesaurus to the data
kex_hvac.fit(vocab='hvac_vocab.csv')
kex_hvac.vocab.head()

,NE,alias,note
token,,,
room,I,room,NaN
replace,S,replace,NaN
find,S,find,NaN
check,S,check,NaN
action_take,X,action_take,NaN


In [35]:
# tag the data, saving to a new DF (instead of internally)
df_pred = kex_hvac.transform(save=False)
df_pred.head()

100%|██████████| 16986/16986 [00:43<00:00, 394.17it/s]


,RawText,Items,Problem,Solution,UK_tok
0,charge m_s pm order site aml complete,site,,"complete, preventative_maintenance, order","aml, m_s, charge m_s, charge"
1,charge m_s pm order aml time charge cover pm '...,,,"preventative_maintenance, order","aml, time charge, 's, charge m_s, time, m_s, c..."
2,adjust repai replace freeze_stat reset poc_ed,freeze_stat,,"adjust, reset, replace","poc_ed, repai"
3,reset adjust repair freeze_stat restart poc_ed,freeze_stat,,"adjust, reset, restart, repair",poc_ed
4,rset adjust repair freeze_stat reset_freezestat,freeze_stat,,"adjust, repair",reset_freezestat


In [36]:
# tag-complete work-orders?
print(df_pred[df_pred.UK_tok ==''].shape[0])
df_pred[df_pred.UK_tok ==''].head()

609


,RawText,Items,Problem,Solution,UK_tok
15,too_hot,,too_hot,,
18,too_hot,,too_hot,,
23,too_cold,,too_cold,,
51,reset_fault start_equipment,,,"start, reset",
56,heater not_work gate f.,"heater, gate",broken,,


In [37]:
# tag-empty? 
print(df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].shape[0])
df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].head()

43


,RawText,Items,Problem,Solution,UK_tok
41,cold a004 a005 fr bateman x5580,,,,"a004, fr, a005"
262,fabricate_metal access cover 28 x 38 1/2 hem f...,,,,"fabricated metal, fabricate_metal, 1/2 hem, 38..."
357,wet_carpet rm 114,,,,"wet_carpet, 114"
618,b349-mod-345b b367-mod-363b a368-mod-362b,,,,
633,cut_hole latch plate const_shop cut_hole latch...,,,,"cut_hole, latch plate, plate, construction_sho..."


## PSU Data, no tagged keywords (yet)

In [4]:
data_directory = os.path.join('./data', 'psu_data')
nl_col = {
    "ProblemDesc": 11,
    "Prob_cause": 12,
    "RepRes": 19
}

#TODO
# meta_col = {

# }

kex_psu = KeywordExtractor('psu_data.xlsx',
                       nlp_cols=nl_col,
#                       meta_cols = meta_col,
#                        special_replace={' -': '; '},
                       wdir=data_directory,
                       keep_temp_files=False
                      )

330it [00:00, 3288.79it/s]

None
making new file...


8887it [00:01, 5208.85it/s]


making new file...


/home/tbsexton/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
672it [00:00, 6716.21it/s]

making new file...


8887it [00:01, 6970.86it/s]


Replacing NaN
Replacing NaN
Replacing NaN


In [6]:
kex_psu.df.head()

,ProblemDesc,Prob_cause,RepRes,RawText
0,Vacuum pump is leaking oil.,add 500 ml fomblin,Replaced Vacuum Pump,vacuum_pump leak oil add 500_ml fomblin replac...
1,Airflow readings range from 50 FPM to 85FPM. ...,add 500 ml fomblin,Adjust Airspeed,airflow reading range 50 fpm 85fpm may_need he...
2,Bottom shelf Connection #4 (marked) Wrap conne...,adjust printhead pressure screws,,bottom_shelf connection 4 mark wrap connection...
3,Camera on autosaw needs cleaned. Having proble...,verify machine operation and perform leak test,,camera autosaw need clean problem pick_up alig...
4,pump at encapsulater is low on oil,Mims wasn't hooked up at time for calibration ...,Added Oil,pump encapsulater low oil mim hook_up time cal...


In [7]:
kex_psu.gen_vocab('psu_vocab.csv', notes=True)

## Automotive data, no tagged keywords (yet)

In [38]:
data_directory = os.path.join('./data', 'auto_data')

nl_col = {
    "DESCRIPTION": 5
}

#TODO
# meta_col = {

# }

kex_auto = KeywordExtractor('auto_raw.xlsx',
                       nlp_cols=nl_col,
#                       meta_cols = meta_col,
#                       special_replace={'Action Taken:': ''},
                       wdir=data_directory,
                       keep_temp_files=True
                      )

1175it [00:00, 11748.19it/s]

None
making new file...


67217it [00:09, 7332.01it/s] 


making new file...


/home/tbsexton/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
1174it [00:00, 11739.59it/s]

...

29877it [00:04, 12400.47it/s]

Replacing NaN
Replacing NaN

39783it [00:05, 10735.16it/s]


Replacing NaN
Replacing NaN
Replacing NaN


67217it [00:09, 6889.02it/s] 


In [39]:
# use this to make a properly formatted thesaurus file for human-annotation
kex_auto.gen_vocab('auto_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv
